In [14]:
%run Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [15]:
isigma = 2
N_centr = 4
N_hists = 4
centralities = ["0-20%","20-40%","40-60%","60-88%"]
centrality = [0,9,0,1,2,3,4,5,6,9]
colors=[1,2,4,root.kGreen+2]
file_path="input/"
file_names=["HeAu_sim_hists_orig","HeAu_sim_hists_embed_","HeAu_sim_hists_6M_","HeAu_sim_hists_6M_sigma_","HeAu_sim_hists_6M_emcid_"]
histo_names = ["hist_pt_orig","hist_inv_mass_FG","hist_inv_mass_BG"]

In [16]:
hists_pi0_FG_read, hists_pi0_BG_read = [], []
hist_stat_read=[]

infile = root.TFile.Open(file_path+file_names[0]+".root", "read")
hist_ref_stat, hist_ref_FG, hist_ref_BG = infile.Get(histo_names[0]+f"{0}"), infile.Get(histo_names[1]+f"{0}"), infile.Get(histo_names[2]+f"{0}")
hist_ref_stat.SetDirectory(root.nullptr)
hist_ref_FG.SetDirectory(root.nullptr)
hist_ref_BG.SetDirectory(root.nullptr)


for ihist in range(N_hists):
    hists_pi0_FG_read_centr, hists_pi0_BG_read_centr = [], []
    hist_stat_centr = []
    for icent in range(N_centr):
        infile = root.TFile.Open(file_path+file_names[1+ihist]+f"{icent}.root", "read")
        print(ihist, icent, file_path+file_names[1+ihist]+f"{icent}.root", histo_names[1]+f"{1+ihist*4+icent}")
        hist_FG = infile.Get(histo_names[1]+f"{1+ihist*4+icent}")
        hist_FG.SetDirectory(root.nullptr)
        hist_BG = infile.Get(histo_names[2]+f"{1+ihist*4+icent}")
        hist_BG.SetDirectory(root.nullptr)
        hist_stat = infile.Get(histo_names[0]+f"{1+ihist*4+icent}")
        hist_stat.SetDirectory(root.nullptr)
        
        hists_pi0_FG_read_centr.append(hist_FG)
        hists_pi0_BG_read_centr.append(hist_BG)
        hist_stat_centr.append(hist_stat)
        infile.Close()
    hists_pi0_FG_read.append(hists_pi0_FG_read_centr)
    hists_pi0_BG_read.append(hists_pi0_BG_read_centr)
    hist_stat_read.append(hist_stat_centr)

0 0 input/HeAu_sim_hists_embed_0.root hist_inv_mass_FG1
0 1 input/HeAu_sim_hists_embed_1.root hist_inv_mass_FG2
0 2 input/HeAu_sim_hists_embed_2.root hist_inv_mass_FG3
0 3 input/HeAu_sim_hists_embed_3.root hist_inv_mass_FG4
1 0 input/HeAu_sim_hists_6M_0.root hist_inv_mass_FG5
1 1 input/HeAu_sim_hists_6M_1.root hist_inv_mass_FG6
1 2 input/HeAu_sim_hists_6M_2.root hist_inv_mass_FG7
1 3 input/HeAu_sim_hists_6M_3.root hist_inv_mass_FG8
2 0 input/HeAu_sim_hists_6M_sigma_0.root hist_inv_mass_FG9
2 1 input/HeAu_sim_hists_6M_sigma_1.root hist_inv_mass_FG10
2 2 input/HeAu_sim_hists_6M_sigma_2.root hist_inv_mass_FG11
2 3 input/HeAu_sim_hists_6M_sigma_3.root hist_inv_mass_FG12
3 0 input/HeAu_sim_hists_6M_emcid_0.root hist_inv_mass_FG13
3 1 input/HeAu_sim_hists_6M_emcid_1.root hist_inv_mass_FG14
3 2 input/HeAu_sim_hists_6M_emcid_2.root hist_inv_mass_FG15
3 3 input/HeAu_sim_hists_6M_emcid_3.root hist_inv_mass_FG16


In [17]:
pt = array( 'd' )
pt_boarders = array( 'd', [1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,6.0,8.0,10.,20.])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
pt_boarders.append(20.)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  array('d', [1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0]) 
pt_boarders =  array('d', [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 8.0, 10.0, 20.0, 20.0])


In [18]:
def emc_fitter(InHistFG, InHistBG, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=1
    seed0 = seed
    seed=seed*N_pt

    hist_pt_bin = InHistFG.ProjectionY("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        pi0_FG_porj = InHistFG.ProjectionX(f"FG{seed*N_pt+p}",hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        pi0_BG_porj = InHistBG.ProjectionX(f"BG{seed*N_pt+p}",hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)

        left = pi0_FG_porj.FindBin(0.4)
        right = pi0_FG_porj.FindBin(0.6)

        pi0_BG_porj.Scale(pi0_FG_porj.Integral(left,right)/(pi0_BG_porj.Integral(left,right)+.1))
        pi0_FG_porj.Add(pi0_BG_porj,-1)
        pi0_FG_porj.SetAxisRange(0.04,0.35)
        pi0_FG_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus{seed+p}', 'gaus',  0.05,0.2)
        g_pol = root.TF1( f'g_pol{seed+p}', '[0]+[1]*x+[2]*x*x',  0.05,0.3)
        Format_Func(g_gaus,3,4)
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}', 'gaus(0)+[3]+[4]*x+[5]*x*x',0.05,0.3)
        if seed<1 and p<2: total = root.TF1( f'total_pos{seed+p}', 'gaus(0)+[3]+[4]*x+[5]*x*x',0.08,0.22)
        Format_Func(total,2,1)


        if fits_mean!=0 and fits_sigma!=0:
            mean_ii=fits_sigma.Eval(pt[p])
            sigma_ii=fits_sigma.Eval(pt[p])
            g_gaus.SetParameter(1,sigma_ii)
            g_gaus.SetParameter(2,sigma_ii)
            g_gaus.SetParLimits(1,mean_ii*0.8-0.05,mean_ii*1.2+0.05)
            g_gaus.SetParLimits(2,sigma_ii*0.5,sigma_ii*1.5)
        else:
            g_gaus.SetParameter(1,0.137)
            g_gaus.SetParameter(2,0.009)
            g_gaus.SetParLimits(1,0.135,0.142)
            g_gaus.SetParLimits(2,0.007,0.02)


        #emc_var_porj.GetXaxis().SetRangeUser(-1.4, 0.5);
        pi0_FG_porj.Draw()
        pi0_FG_porj.Fit( g_gaus, 'QR')
        pi0_FG_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[2]

        total.SetParameters( par_tot )
                
        pi0_FG_porj.Draw()
        pi0_FG_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]

        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1))
        Sigma_local_err.append(total.GetParError(2))

        mean_int = 0.138+0*total.GetParameter(1)
        sigma_int = 0.025+0*abs(total.GetParameter(2))*2

        left_int = pi0_FG_porj.FindBin(mean_int-sigma_int)
        right_int = pi0_FG_porj.FindBin(mean_int+sigma_int)+1
        
        FG, BG = 0.1, 0.1
        for i in range(left_int,right_int):
            FG += pi0_FG_porj.GetBinContent(i) - g_pol.Eval(pi0_FG_porj.GetBinCenter(i))
            BG += pi0_BG_porj.GetBinContent(i)
        
        int_fg_local.append(FG)
        int_bg_local.append(BG)          

        m2_name="fits/"+str(seed0)+"/sigma_emc_"+str(round(pt[p],1))

        if fits_mean==0:
            c1.SaveAs( "output/" + m2_name + ".png" )
        else:
            c1.SaveAs( "output/" + m2_name + "upd.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [19]:
hist_FG, hist_BG, hist_stat =  [hist_ref_FG], [hist_ref_BG], [hist_ref_stat]
for i in range(N_hists):
    for j in range(N_centr):
        hist_FG.append(hists_pi0_FG_read[i][j])
        hist_BG.append(hists_pi0_BG_read[i][j])
        hist_stat.append(hist_stat_read[i][j])

In [20]:
import multiprocess as mp

Ntr = 17

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_fitter, [(hist_FG[i], hist_BG[i], i, isigma) for i in range(Ntr)])
pool.close()

In [21]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for i in range(Ntr):
    Means.append(output_array[i][0])
    Mean_errs.append(output_array[i][1])
    Sigmas.append(output_array[i][2])
    Sigma_errs.append(output_array[i][3])
    Int_FG.append(output_array[i][4])
    Int_BG.append(output_array[i][5])


In [22]:
h2=[]
c2 = root.TCanvas(f"c2",f"c2",500*2,500*2)
c2.Divide(2,2)
g_sigma = []
names = ["embedding","charged veto","sigma veto","emcid"]
legends =[]
for itype in range(Ntr):
    if itype>0 and itype<5: 
        c2.cd(1)
        if itype==1:
            legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype%4]))
            h2.append(Format_Pad_old( 0.05, 16, 0.005, 0.015, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
            h2[-1].Draw()
    if itype>4 and itype<9: 
        c2.cd(2)
        if itype==5:
            legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype%4]))
            h2.append(Format_Pad_old( 0.05, 16, 0.005, 0.015, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
            h2[-1].Draw()
    if itype>8 and itype<13: 
        c2.cd(3)
        if itype==9:
            legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype%4]))
            h2.append(Format_Pad_old( 0.05, 16, 0.005, 0.015, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
            h2[-1].Draw()
    if itype>12: 
        c2.cd(4)
        if itype==13:
            legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype%4]))
            h2.append(Format_Pad_old( 0.05, 16, 0.005, 0.015, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
            h2[-1].Draw()

    g_sigma.append(root.TGraphErrors(N_pt, pt, Sigmas[itype],pt_err,Sigma_errs[itype]))
               
    Format_Graph(g_sigma[-1],20,1, colors[itype%4], 3, colors[itype%4],1,1)
    if itype==0: Format_Graph(g_sigma[0],20,1, 6, 3, 6,1,1)
    g_sigma[-1].Draw('P')
    
    if itype>0:
        legends[-1].AddEntry(g_sigma[-1],centralities[itype%4],"p")
        if itype%4==0:
            g_sigma[0].Draw('P')
            legends[-1].SetFillColorAlpha(0,0)
            legends[-1].SetLineWidth(0)
            legends[-1].AddEntry(g_sigma[0],"default","p")    
            legends[-1].Draw()

c2.Draw()
c2.SaveAs("output/pi0_sigma.png")

In [23]:
h3=[]
c3 = root.TCanvas(f"c2",f"c2",500*2,500*2)
c3.Divide(2,2)
g_sigma = []
names = ["embedding","charged veto","sigma veto","emcid"]
legends =[]
for itype in range(Ntr):
    if itype>0 and itype<5: 
        c3.cd(1)
        if itype==1:
            legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype%4]))
            h3.append(Format_Pad_old( 0.05, 16, 0.13, 0.15, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
            h3[-1].Draw()
    if itype>4 and itype<9: 
        c3.cd(2)
        if itype==5:
            legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype%4]))
            h3.append(Format_Pad_old( 0.05, 16, 0.13, 0.15, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
            h3[-1].Draw()
    if itype>8 and itype<13: 
        c3.cd(3)
        if itype==9:
            legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype%4]))
            h3.append(Format_Pad_old( 0.05, 16, 0.13, 0.15, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
            h3[-1].Draw()
    if itype>12: 
        c3.cd(4)
        if itype==13:
            legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype%4]))
            h3.append(Format_Pad_old( 0.05, 16, 0.13, 0.15, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
            h3[-1].Draw()

    g_sigma.append(root.TGraphErrors(N_pt, pt, Means[itype],pt_err,Mean_errs[itype]))
               
    Format_Graph(g_sigma[-1],20,1, colors[itype%4], 3, colors[itype%4],1,1)
    if itype==0: Format_Graph(g_sigma[0],20,1, 6, 3, 6,1,1)
    g_sigma[-1].Draw('P')
    
    if itype>0:
        legends[-1].AddEntry(g_sigma[-1],centralities[itype%4],"p")
        if itype%4==0:
            g_sigma[0].Draw('P')
            legends[-1].SetFillColorAlpha(0,0)
            legends[-1].SetLineWidth(0)
            legends[-1].AddEntry(g_sigma[0],"default","p")    
            legends[-1].Draw()

c3.Draw()
c3.SaveAs("output/pi0_mean.png")

In [24]:
FG_BG, FG_BG_err = [], []
for icase in range(Ntr-1):
    FG_BG_loc =  array( 'd' )
    FG_BG_err_loc =  array( 'd' )
    for ipt in range(N_pt):
            if icase<9: FG_BG_loc.append(Int_FG[icase+1][ipt]/(Int_FG[0][ipt]+.1)/(hist_stat[icase+1].GetEntries()/hist_stat[0].GetEntries()))
            else: FG_BG_loc.append(Int_FG[icase+1][ipt]/(Int_FG[8][ipt]+.1)/(hist_stat[icase+1].GetEntries()/hist_stat[8].GetEntries()))
            FG_BG_err_loc.append(FG_BG_loc[-1]*0.005)
        
    FG_BG.append(FG_BG_loc)
    FG_BG_err.append(FG_BG_err_loc)

In [25]:
h4=[]
c4 = root.TCanvas(f"c4",f"c4",500*2,500*2)
c4.Divide(2,2)
g_sigma = [[] for _ in range(Ntr)]
g_sigma_inter = [[] for _ in range(Ntr)]
names = ["emc_r_sdphi","emc_min_r_sdphi","emc_cnt_sdphi","emc_r_sdz","emc_min_r_sdz","emc_cnt_sdz"]
legends =[]


for itype in range(Ntr-1):
    if itype<4: 
        c4.cd(1)
        if itype==0:
            h4.append(Format_Pad_old( 0.05, 16, 0.9, 1.05, "p_{T} (GeV/#it{c})", "Cut/Default", 1.1, 1.1, 0.03, 0.03, "#pi^{0} Sim" ))
            h4[-1].Draw()
            legends.append(root.TLegend(0.5,0.1,0.9,0.5,"embedding"))
    elif itype<8: 
        c4.cd(2)
        if itype==4:
            h4.append(Format_Pad_old( 0.05, 16, 0.9, 1.05, "p_{T} (GeV/#it{c})", "Cut/Default", 1.1, 1.1, 0.03, 0.03, "#pi^{0} Sim" ))
            h4[-1].Draw()
            legends.append(root.TLegend(0.5,0.1,0.9,0.5,"charged veto"))
    elif itype<12: 
        c4.cd(3)
        if itype==8:
            h4.append(Format_Pad_old( 0.05, 16, 0.5, 1.05, "p_{T} (GeV/#it{c})", "Cut/Default", 1.1, 1.1, 0.03, 0.03, "#pi^{0} Sim" ))
            h4[-1].Draw()
            legends.append(root.TLegend(0.5,0.1,0.9,0.5,"sigma veto"))
    else: 
        c4.cd(4)
        if itype==12:
            h4.append(Format_Pad_old( 0.05, 16, 0.7, 1.05, "p_{T} (GeV/#it{c})", "Cut/Default", 1.1, 1.1, 0.03, 0.03, "#pi^{0} Sim" ))
            h4[-1].Draw()
            legends.append(root.TLegend(0.5,0.1,0.9,0.5,"emcid veto"))
    
    
   
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)

    g_sigma[itype].append(root.TGraphErrors(N_pt, pt, FG_BG[itype],pt_err,FG_BG_err[itype]))
               
    Format_Graph(g_sigma[itype][-1],20,1, colors[itype%4], 3, colors[itype%4],1,1)
    g_sigma[itype][-1].Draw('P')
      
    legends[-1].AddEntry(g_sigma[itype][-1],centralities[itype%4],"p")
        
    legends[-1].Draw()
c4.Draw()
c4.SaveAs("output/pi0_FG.png")

In [26]:
#!jupyter nbconvert --to html emc_r_HeAu.ipynb --HTMLExporter.theme=dark;